# Application-to-Permit in New York City

This notebook (along with files associated with it) examines the timeline of permit application to permit issuance in New York City.

## Introduction

### Construction Permit Process

Before a new building can begin construction it has to go through a long professional vetting process. The Department of Buildings (DOB) is the principal governmental organization responsible for overseeing the inspections and paperwork required for breaking ground.

The exact process, for the building manager involved, is summarized by [this handy chart](http://www1.nyc.gov/site/buildings/homeowner/dob-forms-applications.page). Here are the steps:

1. The applicant prefiles the application. Prefiling saves time, but not all applicants invoke it.
2. The applicant submits a work plan.
3. The plan is either professionally certified (a so-called "pro cert" or "self cert") or reviewed by the DOB.
4. The plan is approved or denied by the DOB.
5. Once a plan is approved, the applicant requests a work permit.
6. DOB issues a work permit.
7. Construction begins.

### Retrieving Construction Permits

Construction permits are retrieved as a part of the work done by the `Permit Issuance` companion notebook, which came first.

### Retrieving Application Jobs

DOB publishes a list of all [building job applications](https://data.cityofnewyork.us/Housing-Development/DOB-Job-Application-Filings/ic3t-wcy2) as a part of the New York City Open Data Portal. This application is missing new building construction (which is posted to a seperate open dataset), but contains all of the information on the preceding timeline. Retrieving these permits is merely a matter of accessing, extracting, and filtering this dataset.

## Reading Job Application Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
permits = pd.read_csv("DOB_Job_Application_Filings.csv")

/Users/Honors/anaconda/envs/nyc-construction-timelines/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (13,14,16,19,38,59,60,78,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pd.set_option("max_columns", 500)

In [4]:
permits.head(1)

,Job #,Doc #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Job Status,Job Status Descrp,Latest Action Date,Building Type,Community - Board,Cluster,Landmarked,Adult Estab,Loft Board,City Owned,Little e,PC Filed,eFiling Filed,Plumbing,Mechanical,Boiler,Fuel Burning,Fuel Storage,Standpipe,Sprinkler,Fire Alarm,Equipment,Fire Suppression,Curb Cut,Other,Other Description,Applicant's First Name,Applicant's Last Name,Applicant Professional Title,Applicant License #,Professional Cert,Pre- Filing Date,Paid,Fully Paid,Assigned,Approved,Fully Permitted,Initial Cost,Total Est. Fee,Fee Status,Existing Zoning Sqft,Proposed Zoning Sqft,Horizontal Enlrgmt,Vertical Enlrgmt,Enlargement SQ Footage,Street Frontage,ExistingNo. of Stories,Proposed No. of Stories,Existing Height,Proposed Height,Existing Dwelling Units,Proposed Dwelling Units,Existing Occupancy,Proposed Occupancy,Site Fill,Zoning Dist1,Zoning Dist2,Zoning Dist3,Special District 1,Special District 2,Owner Type,Non-Profit,Owner's First Name,Owner's Last Name,Owner's Business Name,Owner's House Number,Owner'sHouse Street Name,City,State,Zip,Owner'sPhone #,Job Description,DOBRunDate
0,121577873,2,MANHATTAN,386,PARK AVENUE SOUTH,857,38.0,1016890.0,A2,D,APPLICATION PROCESSED - ENTIRE,04/25/2013,OTHER,105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,X,X,,,,,,,,,,,,DOUGLAS,MASS,PE,0058375,NaN,04/25/2013,04/25/2013,04/25/2013,NaN,NaN,NaN,$75000.00,$986.00,STANDARD,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JAMES,MIGLIORE,MACKLOWE MANAGEMENT,126,EAST 56TH STREET,NEW YORK,NY,10222,2125545837,GENERAL MECHANICAL & PLUMBING MODIFICATIONS AS...,04/26/2013 12:00:00 AM


In [5]:
permits['Job Status Descrp'].value_counts()[:-1] # Last one is a parsing error.

APPLICATION PROCESSED - ENTIRE              60217
PLAN EXAM - APPROVED                        36135
APPLICATION PROCESSED - ENTIRE              26308
PLAN EXAM - IN PROCESS                      23352
PLAN EXAM - APPROVED                        13840
PERMIT ISSUED - ENTIRE JOB/WORK             11975
PLAN EXAM - DISAPPROVED                      9453
PLAN EXAM - IN PROCESS                       7559
PERMIT ISSUED - ENTIRE JOB/WORK              7397
PRE-FILING                                   7052
PERMIT ISSUED - PARTIAL JOB                  5498
APPLICATION ASSIGNED TO PLAN EXAMINER        5216
PRE-FILING                                   3864
APPLICATION PROCESSED - NO PLAN EXAM         3554
PLAN EXAM - DISAPPROVED                      3291
PERMIT ISSUED - PARTIAL JOB                  2358
APPLICATION ASSIGNED TO PLAN EXAMINER        1411
APPLICATION PROCESSED - NO PLAN EXAM         1266
APPLICATION PROCESSED-PART-NO PAYMENT         378
APPLICATION PROCESSED-PART-NO PAYMENT         175


Records begin in time at 04/25/2013.

This record includes submissions for all job types, however we're only interested in new building construction jobs. This corresponds to records with a `JOB TYPE` of `NB`.

In [25]:
new_buildings = permits[permits['Job Type'] == 'NB']

In [26]:
len(new_buildings)

12481

In [27]:
all_bins = np.unique(new_buildings['Bin #'])

In [29]:
len(all_bins)

6951

In [36]:
new_buildings['Job Status Descrp'].value_counts()

APPLICATION PROCESSED - ENTIRE              4850
APPLICATION ASSIGNED TO PLAN EXAMINER       1860
APPLICATION PROCESSED - ENTIRE              1748
PRE-FILING                                  1712
PRE-FILING                                   751
APPLICATION ASSIGNED TO PLAN EXAMINER        553
PLAN EXAM - IN PROCESS                       351
PLAN EXAM - DISAPPROVED                      329
PLAN EXAM - DISAPPROVED                       96
PLAN EXAM - IN PROCESS                        80
PLAN EXAM - APPROVED                          76
PLAN EXAM - APPROVED                          45
APPLICATION PROCESSED-PART-NO PAYMENT         18
APPLICATION PROCESSED-PART-NO PAYMENT          6
PERMIT ISSUED - ENTIRE JOB/WORK                5
Name: Job Status Descrp, dtype: int64

In [37]:
permits['Job Status Descrp'].value_counts()[:-1] / len(permits)

APPLICATION PROCESSED - ENTIRE              0.261133
PLAN EXAM - APPROVED                        0.156701
APPLICATION PROCESSED - ENTIRE              0.114085
PLAN EXAM - IN PROCESS                      0.101267
PLAN EXAM - APPROVED                        0.060018
PERMIT ISSUED - ENTIRE JOB/WORK             0.051930
PLAN EXAM - DISAPPROVED                     0.040993
PLAN EXAM - IN PROCESS                      0.032780
PERMIT ISSUED - ENTIRE JOB/WORK             0.032077
PRE-FILING                                  0.030581
PERMIT ISSUED - PARTIAL JOB                 0.023842
APPLICATION ASSIGNED TO PLAN EXAMINER       0.022619
PRE-FILING                                  0.016756
APPLICATION PROCESSED - NO PLAN EXAM        0.015412
PLAN EXAM - DISAPPROVED                     0.014272
PERMIT ISSUED - PARTIAL JOB                 0.010226
APPLICATION ASSIGNED TO PLAN EXAMINER       0.006119
APPLICATION PROCESSED - NO PLAN EXAM        0.005490
APPLICATION PROCESSED-PART-NO PAYMENT       0.

In [38]:
new_buildings['Job Status Descrp'].value_counts() / len(new_buildings)

APPLICATION PROCESSED - ENTIRE              0.388591
APPLICATION ASSIGNED TO PLAN EXAMINER       0.149027
APPLICATION PROCESSED - ENTIRE              0.140053
PRE-FILING                                  0.137168
PRE-FILING                                  0.060171
APPLICATION ASSIGNED TO PLAN EXAMINER       0.044307
PLAN EXAM - IN PROCESS                      0.028123
PLAN EXAM - DISAPPROVED                     0.026360
PLAN EXAM - DISAPPROVED                     0.007692
PLAN EXAM - IN PROCESS                      0.006410
PLAN EXAM - APPROVED                        0.006089
PLAN EXAM - APPROVED                        0.003605
APPLICATION PROCESSED-PART-NO PAYMENT       0.001442
APPLICATION PROCESSED-PART-NO PAYMENT       0.000481
PERMIT ISSUED - ENTIRE JOB/WORK             0.000401
PERMIT ISSUED - PARTIAL JOB                 0.000080
Name: Job Status Descrp, dtype: float64

In [39]:
new_buildings['Job Status Descrp'].value_counts()[['PERMIT ISSUED - ENTIRE JOB/WORK', 'PERMIT ISSUED - PARTIAL JOB']]

PERMIT ISSUED - ENTIRE JOB/WORK    5
PERMIT ISSUED - PARTIAL JOB        1
Name: Job Status Descrp, dtype: int64

In [34]:
permits['Job Type'].value_counts()

NB    12481
Name: Job Type, dtype: int64